# Agent Based Models

In [1]:
!pip install simpy

## Simpy: What we have learned so far

* Simpy provides a simulation environment that manages discrete-event simulations
* Use Python generators to model behavior of components of a larger system.
* Register instances of generators with the simulation environment
* Append "who, what, value" to a data log for post-processing 


What we will learn in this unit:

* Modeling multiple units
* Modeling a shared resource
* Extracting data from a data log


## Example: A room full of Roombas

Let's imagine a large facility that is being cleaned by a collection of Roomba-type robotic cleaning units.  Each unit is characterized by time required to charge, and an amount of time it can clean before needing to be recharged. The facility must be cleaned during a 16 hour overnight shift. On average, 3 units must be operating continuously to meet the cleaning requirements, i.e., 3 x 16 = 48 hours machine cleaning each night. We would like to determine how many charging stations will be required.

| Unit | Charge Time (hrs) | Clean Time (hrs) |
| :--: | :--: | :--: |
| A | 1.0 | 2.5 |
| B | 0.5 | 1.5 |
| C | 0.8 | 2.0 |
| D | 1.4 | 3.5 |
| E | 0.5 | 1.2 |

![roomba](https://upload.wikimedia.org/wikipedia/commons/2/27/%D0%A0%D0%BE%D0%B1%D0%BE%D1%82_%D0%BF%D1%8B%D0%BB%D0%B5%D1%81%D0%BE%D1%81_Roomba_780.jpg)

In [108]:
roomba_data = [
    ["A", 1.0, 2.5],
    ["B", 0.5, 1.5],
    ["C", 0.8, 2.0],
    ["D", 1.4, 3.5],
    ["E", 0.5, 1.2],
]

roomba_df = pd.DataFrame(roomba_data, columns=["id", "charge_time", "clean_time"])
roomba_df

,id,charge_time,clean_time
0,A,1.0,2.5
1,B,0.5,1.5
2,C,0.8,2.0
3,D,1.4,3.5
4,E,0.5,1.2


## One Roomba

The first challenge is to model the performance of a single Roomba. Our first attempt at a model consists of a simple Python generator. The data log consists of start and finish of each charge and cleaning cycle. For this first attempt, we'll assume a charging station is always available when needed.

In [91]:
import simpy 
import pandas as pd

data_log = []

def roomba_model(id, charge_time, clean_time):
    while True:
        tic = env.now
        yield env.timeout(charge_time)
        toc = env.now
        data_log.append([id, "charging", tic, toc])
   
        tic = env.now
        yield env.timeout(clean_time)
        toc = env.now
        data_log.append([id, "cleaning", tic, toc])
        
env = simpy.Environment()

roomba = roomba_model("A", 1.0, 2.5)

env.process(roomba)

env.run(until=16)
df = pd.DataFrame(data_log, columns=["id", "event", "begin", "end"])
display(df)

,id,event,begin,end
0,A,charging,0.0,1.0
1,A,cleaning,1.0,3.5
2,A,charging,3.5,4.5
3,A,cleaning,4.5,7.0
4,A,charging,7.0,8.0
5,A,cleaning,8.0,10.5
6,A,charging,10.5,11.5
7,A,cleaning,11.5,14.0
8,A,charging,14.0,15.0


## Extracting data from the data log

Let's extract some performance indicators from the data log.

In [95]:
# extract list of unique roombas
roombas = list(set(df["id"]))
print(roombas)

['A']


In [104]:
# time spent charging
for r in roombas:
    idx = (df["id"]==r) & (df["event"]=="charging")
    dt = df[idx]["end"] - df[idx]["begin"]
    t_charge = sum(dt)
    print(r, "charge time =", t_charge)
    
for r in roombas:
    idx = (df["id"]==r) & (df["event"]=="cleaning")
    dt = df[idx]["end"] - df[idx]["begin"]
    t_clean = sum(dt)
    print(r, "clean time =", t_clean)

A charge time = 5.0
A clean time = 10.0


## Adding Roombas

In [177]:
import simpy 
import pandas as pd

data_log = []

def roomba_model(id, charge_time, clean_time):
    while True:
        tic = env.now
        yield env.timeout(charge_time)
        toc = env.now
        data_log.append([id, "charging", tic, toc])
   
        tic = env.now
        yield env.timeout(clean_time)
        toc = env.now
        data_log.append([id, "cleaning", tic, toc])
        
env = simpy.Environment()

for r in roomba_df.index:
    env.process(roomba_model(roomba_df["id"][r], roomba_df["charge_time"][r], roomba_df["clean_time"][r]))

env.run(until=16)
df = pd.DataFrame(data_log, columns=["id", "event", "begin", "end"])
display(df)

,id,event,begin,end
0,B,charging,0.0,0.5
1,E,charging,0.0,0.5
2,C,charging,0.0,0.8
3,A,charging,0.0,1.0
4,D,charging,0.0,1.4
5,E,cleaning,0.5,1.7
6,B,cleaning,0.5,2.0
7,E,charging,1.7,2.2
8,B,charging,2.0,2.5
9,C,cleaning,0.8,2.8


## Extracting data from data logs

### Writing your own methods

In [178]:
# time spent charging
roombas = list(set(df["id"]))
roombas.sort()
for r in roombas:
    idx = (df["id"]==r) & (df["event"]=="charging")
    dt = df[idx]["end"] - df[idx]["begin"]
    t_charge = sum(dt)
    print(r, "charge time =", t_charge)

print()

for r in roombas:
    idx = (df["id"]==r) & (df["event"]=="cleaning")
    dt = df[idx]["end"] - df[idx]["begin"]
    t_clean = sum(dt)
    print(r, "clean time =", t_clean)

A charge time = 5.0
B charge time = 4.0
C charge time = 4.800000000000002
D charge time = 4.200000000000001
E charge time = 5.0

A clean time = 10.0
B clean time = 10.5
C clean time = 10.0
D clean time = 10.5
E clean time = 10.799999999999997


### Cut and Paste into Google Sheets

### Using Pandas pivot tables

Using Pandas pivot table

In [181]:
import numpy as np

df["time"] = df["end"] - df["begin"]
pd.pivot_table(df, index=["id", "event"], values="time", aggfunc={"time":np.sum} )

time
id event         
A  charging   5.0
   cleaning  10.0
B  charging   4.0
   cleaning  10.5
C  charging   4.8
   cleaning  10.0
D  charging   4.2
   cleaning  10.5
E  charging   5.0
   cleaning  10.8

## Introducing Shared Resources

A charging station is an example of a **shared resource**. There are three types of resources that can be modeled in Simpy:

* **Resource** Resources that can only used by a limited number of processes at a time.
* **Stores** Resources that can store or release Python objects.
* **Containers** Resources that model the production and consumption of bulk goods.

In this example, charging stations are an example of `Resources`. 

The first step is to add the resource to the model. Resources are added as Simpy `Resource` object.  The user of the resource must request and release the resource.

In [174]:
import simpy 
import pandas as pd

data_log = []

def roomba_model(id, charge_time, clean_time):
    while True:
        request = chargers.request()
        yield request
        tic = env.now
        yield env.timeout(charge_time)
        chargers.release(request)
        toc = env.now
        data_log.append([id, "charging", tic, toc])
   
        tic = env.now
        yield env.timeout(clean_time)
        toc = env.now
        data_log.append([id, "cleaning", tic, toc])
        
env = simpy.Environment()
chargers = simpy.Resource(env, capacity=1)

for r in roomba_df.index:
    env.process(roomba_model(roomba_df["id"][r], roomba_df["charge_time"][r], roomba_df["clean_time"][r]))

env.run(until=16)
df = pd.DataFrame(data_log, columns=["id", "event", "begin", "end"])
display(df)

,id,event,begin,end
0,A,charging,0.0,1.0
1,B,charging,1.0,1.5
2,C,charging,1.5,2.3
3,B,cleaning,1.5,3.0
4,A,cleaning,1.0,3.5
5,D,charging,2.3,3.7
6,E,charging,3.7,4.2
7,C,cleaning,2.3,4.3
8,B,charging,4.2,4.7
9,E,cleaning,4.2,5.4


In [176]:
df["time"] = df["end"] - df["begin"]
pd.pivot_table(df, index=["event"], values="time", aggfunc={"time":np.sum} )

,time
event,
charging,15.4
cleaning,31.3


## Assignment for Thursday

1. Determine the minimum number of chargers required to meet cleaning requirement.
2. Modify the model to assume the changers are fully charged at the start of the cleaning shift. Does that reduce the number of chargers required?
3. If you have time: Create a chart showing the utilization of each Roomba ... the time it is charging, cleaning, and remaining idle.